<a href="https://colab.research.google.com/github/nageshandino/explainers/blob/main/Simulated_Annealing_simple_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pulp
# Example of simulated annealing for a simple operations research problem.
# This demo shows that simulated annealing can achieve optimal solution but may NOT always achieve optimal solution, and it is sensitive to different settings.

# In this example
# 1) all orders start at same fulfillment center
# 2) there are M carriers
# 3) there are N customer destinations
# 4) Cost from fulfillment center is C(i, j) is how much carrier number j charges for destination i,
    # 1<=i<=M, 1<<=j<=N,
    # C(i,j) is infinity if a destination is not supported by a carrier
# 5) Optimization problem is - minimize total cost of fulfillment

from pulp import *
import numpy as np
import random
import math
import matplotlib.pyplot as plt

print(f"\n\n===This demo shows that simulated annealing does NOT always achieve optimal solution, and it is sensitive to different settings===")

# Initial solution - Randomly assign orders to carriers.
def generate_initial_solution(N, M):
    return [random.randint(0, M-1) for _ in range(N)]

# Calculate total cost given a solution and cost matrix.
def calculate_cost(solution, cost_matrix):
    total_cost = 0
    for destination, carrier in enumerate(solution):
        if cost_matrix[destination, carrier] == float('inf'):
            return float('inf')
        total_cost += cost_matrix[destination, carrier]
    return total_cost


# Slightly change the current solution, and get to a nearby solution.
# Randomly pick an order-carrier assignment, and change the carrier to a different carrier.
def generate_neighbor(solution, M):
    new_solution = solution.copy()
    index = random.randint(0, len(solution) - 1)
    new_carrier = random.randint(0, M-1)
    while new_carrier == new_solution[index]:
        new_carrier = random.randint(0, M-1)
    new_solution[index] = new_carrier
    return new_solution


# Core annealing
def simulated_annealing(N, M, cost_matrix, initial_temp, cooling_rate, num_iterations):
    current_solution = generate_initial_solution(N, M)
    current_cost = calculate_cost(current_solution, cost_matrix)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temp

    cost_history = [current_cost]

    for i in range(num_iterations):
        neighbor_solution = generate_neighbor(current_solution, M)
        neighbor_cost = calculate_cost(neighbor_solution, cost_matrix)

        if neighbor_cost == float('inf'):
            continue

        cost_diff = neighbor_cost - current_cost

        # This is the crux of annealing, if a perturbed solution has better cost make it current solution.
        # even if perturbed solution has worse cost,
        # we will still accept it if random.random() < math.exp(-cost_diff / temperature)
        # the idea is -> at lower temparatures, you will have less probability of accepting worse solutions
        # **worse case here is new solution has higher cost than existing solution, or cost_diff>0 **
        # accept worse solution if a random number generated between 0 & 1 is less than math.exp(-cost_diff / temperature)
        #       cost diff is -20, temparature is 1000
        #             math.exp(-20/1000) ≈ 0.9802 (98.02%)
        #       cost diff is -20, temparature is 100
        #             math.exp(-20/100) ≈ 0.8187 (81.87%)
        #       cost diff is -20, temparature is 10
        #             math.exp(-20/10) ≈ 0.1353 (13.53%)



        if cost_diff < 0 or random.random() < math.exp(-cost_diff / temperature):
            current_solution = neighbor_solution
            current_cost = neighbor_cost

            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        temperature *= cooling_rate
        cost_history.append(current_cost)

    return best_solution, best_cost, cost_history

# Example usage
N = 10  # Number of destinations
M = 3   # Number of carriers

np.random.seed(42)
# Generate random cost matrix
cost_matrix = np.random.randint(10, 100, size=(N, M)).astype(float)
print("\n\n===Simulated Annealing Approach, experiment with different paramters ===\n\n")
print("Let us run simulated annealing 4 times, each time with different temparature, cooling rate and number of iteration settings")
print("\nAnd observe the solution generated..")

i_t= 1000
n_i = 250
c_r = 0.995
# Run simulated annealing
best_solution, best_cost, cost_history = simulated_annealing(
    N, M, cost_matrix, initial_temp=i_t, cooling_rate=c_r, num_iterations=n_i
)
print(f"\n\nRun 1 - Results with {n_i} iterations & initial temparature={i_t} & cooling rate={c_r}")

print(f"\nSolution cost: {best_cost}")
for carrier in range(M):
    carrier_destinations = [i for i, c in enumerate(best_solution) if c == carrier]
    print(f"Carrier {carrier + 1}: Destinations {carrier_destinations}")


i_t= 1000
n_i = 500
c_r = 0.995
# Run simulated annealing
best_solution, best_cost, cost_history = simulated_annealing(
    N, M, cost_matrix, initial_temp=i_t, cooling_rate=c_r, num_iterations=n_i
)
print(f"\n\nRun 2 - Results with {n_i} iterations & initial temparature={i_t} & cooling rate={c_r}")

print(f"\nSolution cost: {best_cost}")
for carrier in range(M):
    carrier_destinations = [i for i, c in enumerate(best_solution) if c == carrier]
    print(f"Carrier {carrier + 1}: Destinations {carrier_destinations}")

i_t= 1000
n_i = 500
c_r = 0.99
# Run simulated annealing
best_solution, best_cost, cost_history = simulated_annealing(
    N, M, cost_matrix, initial_temp=i_t, cooling_rate=c_r, num_iterations=n_i
)
print(f"\n\nRun 3 - Results with {n_i} iterations & initial temparature={i_t} & cooling rate={c_r}")

print(f"\nSolution cost: {best_cost}")
for carrier in range(M):
    carrier_destinations = [i for i, c in enumerate(best_solution) if c == carrier]
    print(f"Carrier {carrier + 1}: Destinations {carrier_destinations}")

i_t= 1000
n_i = 500
c_r = 0.9
# Run simulated annealing
best_solution, best_cost, cost_history = simulated_annealing(
    N, M, cost_matrix, initial_temp=i_t, cooling_rate=c_r, num_iterations=n_i
)
print(f"\n\nRun 4 - Results with {n_i} iterations & initial temparature={i_t} & cooling rate={c_r}")

print(f"\nSolution cost: {best_cost}")
for carrier in range(M):
    carrier_destinations = [i for i, c in enumerate(best_solution) if c == carrier]
    print(f"Carrier {carrier + 1}: Destinations {carrier_destinations}")

print("\n\n===End of Simulated Annealing Runs ===\n\n")


print("\n\n===Tradtional Mixed Integer Linear Programming Approach===\n\n")


# Generate cost reuse the same cost matrix as above, and same M & N values
C = cost_matrix

# Create the LP problem
prob = LpProblem("Carrier_Assignment", LpMinimize)

# Binary decision variable for N Destinations, M Carriers.
# x is a 10 X 3 matrix that captures assignments
# If x[i,j] is 1, order i is assigned to carrier j

x = [[LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(M)] for i in range(N)]

# Objective function --> (cost from cost matrix) * (assigned or not assigned)
prob += lpSum(C[i][j] * x[i][j] for i in range(N) for j in range(M))

# Constraint --> each order destination must be assigned to exactly one carrier
for i in range(N):
    prob += lpSum(x[i][j] for j in range(M)) == 1

# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print("\nMILP Optimal Assignments:")
total_cost = 0

for i in range(N):
    for j in range(M):
        if value(x[i][j]) == 1:
            print(f"Destination {i} assigned to Carrier {j+1} (Cost: {C[i][j]})")
            total_cost += C[i][j]

print(f"\nMILP Total Cost: {total_cost}")


# print("\n\nCost Matrix (Destinations x Carriers):")
# print(cost_matrix)




===This demo shows that simulated annealing does NOT always achieve optimal solution, and it is sensitive to different settings===


===Simulated Annealing Approach, experiment with different paramters ===


Let us run simulated annealing 4 times, each time with different temparature, cooling rate and number of iteration settings

And observe the solution generated..


Run 1 - Results with 250 iterations & initial temparature=1000 & cooling rate=0.995

Solution cost: 388.0
Carrier 1: Destinations [7, 9]
Carrier 2: Destinations [0, 1, 2, 5, 8]
Carrier 3: Destinations [3, 4, 6]


Run 2 - Results with 500 iterations & initial temparature=1000 & cooling rate=0.995

Solution cost: 363.0
Carrier 1: Destinations [1, 6, 7]
Carrier 2: Destinations [0, 2, 5, 8]
Carrier 3: Destinations [3, 4, 9]


Run 3 - Results with 500 iterations & initial temparature=1000 & cooling rate=0.99

Solution cost: 323.0
Carrier 1: Destinations [6, 7]
Carrier 2: Destinations [0, 1, 5, 8]
Carrier 3: Destinations [2,